# Analyse réseau - AFIR 


## Objectif 

Identification des stations Qualicharge du réseau RTE-T et autoroutier.

*consultation du NoteBook avec visualisation des cartes sur [nbviewer](https://nbviewer.org/github/loco-labs/qualicharge-rtet/blob/main/notebooks/V2_afir.ipynb)*

In [1]:
import json, copy
from shapely import LineString, Point
import numpy as np
import geopandas as gpd
import pandas as pd
import geo_nx as gnx 
import networkx as nx 

from geo_nx import geom_to_crs, cast_id
from fonction_rtet import insertion_noeuds, proximite, insertion_projection, association_stations
from interfaces import creation_pandas_stations
from afir import Afir

data_path = '../data/'
rtet_path = '../rtet/'
refnat = {'tiles': 'cartodbpositron', 'location': [46.3, 2.3], 'zoom_start': 5}

vl_central_2025 = {'central': True, 'pcum': 400,  'pmax': 150, 'dist': 60000}
vl_central_2027 = {'central': True, 'pcum': 600,  'pmax': 150, 'dist': 60000}
vl_global_2027 = {'central': False, 'pcum': 300,  'pmax': 150, 'dist': 60000}
pl_global_2025 = {'central': False, 'pcum': 1400, 'pmax': 350, 'dist': 120000}
pl_central_2027 = {'central': True, 'pcum': 2800, 'pmax': 350, 'dist': 120000}
#pl_global_2027 = pl_global_2025

version = 'V1'

## Chargement des données

- réseau routier RTE-T,
- stations IRVE

In [2]:
gr_rtet = gnx.from_geopandas_edgelist(gpd.read_file(rtet_path + version + '_rtet_edge.geojson'), 
                                 node_gdf=gpd.read_file(rtet_path + version + '_rtet_node.geojson'),
                                 node_attr=True, edge_attr=True)
len(gr_rtet.nodes), len(gr_rtet.edges), 

(2686, 2757)

In [3]:
stations = creation_pandas_stations(data_path + 'export_afir_v2_2024.csv', nature="station_irve", first_id=max(gr_rtet.nodes) + 1, source='qualicharge')
print(len(stations))
print(stations['operateur'].unique())
stations[stations.operateur=='Tesla']

1318
['Power Dot France' 'Bouygues E&S' 'Tesla' 'Fastned France']


,p_cum,p_max,id_station,operateur,amenageur,geometry,node_id,nature
7,3000.0,250.0,FRTSLP29658,Tesla,Tesla,POINT (3854644.511 7280683.72),2856,station_irve
12,520.0,130.0,FRTSLP1754,Tesla,Tesla,POINT (3836698.159 7103046.28),2861,station_irve
17,5000.0,250.0,FRTSLP29903,Tesla,Tesla,POINT (3830241.682 7048719.177),2866,station_irve
19,8000.0,250.0,FRTSLP30379,Tesla,Tesla,POINT (3877933.591 7925055.123),2868,station_irve
20,7000.0,250.0,FRTSLP28778,Tesla,Tesla,POINT (3870154.037 7502409.185),2869,station_irve
...,...,...,...,...,...,...,...,...
1293,3000.0,250.0,FRTSLP30255,Tesla,Tesla,POINT (3856095.186 7297555.085),4142,station_irve
1294,4000.0,250.0,FRTSLP14836,Tesla,Tesla,POINT (3864235.18 7404171.639),4143,station_irve
1303,3000.0,250.0,FRTSLP30237,Tesla,Tesla,POINT (3831717.622 7060810.159),4152,station_irve
1304,1500.0,150.0,FRTSLP6431,Tesla,Tesla,POINT (3864661.365 7410485.457),4153,station_irve


## Intégration des données

### objet de la classe Afir

In [4]:
log=True
vl_g_27 = Afir(gr_rtet, stations, log=log, **vl_global_2027)
vl_c_25 = Afir(gr_rtet, stations, log=log, **vl_central_2025)
vl_c_27 = Afir(gr_rtet, stations, log=log, **vl_central_2027)
pl_g_25 = Afir(gr_rtet, stations, log=log, **pl_global_2025)
pl_c_27 = Afir(gr_rtet, stations, log=log, **pl_central_2027)
scenarios = {'vl_c_25': vl_c_25, 
             'vl_c_27': vl_c_27, 
             'vl_g_27': vl_g_27, 
             'pl_g_25': pl_g_25, 
             'pl_c_27': pl_c_27}

print('nb stations / scenario : ', {key: len(scenar.gs_all) for key, scenar in scenarios.items()})

Nb stations (st, pre_st, ext, out, total) :  0 3 139 347 489
Nb stations (st, pre_st, ext, out, total) :  0 0 68 418 486
Nb stations (st, pre_st, ext, out, total) :  0 0 64 389 453
Nb stations (st, pre_st, ext, out, total) :  0 0 0 0 0
Nb stations (st, pre_st, ext, out, total) :  0 0 0 0 0
nb stations / scenario :  {'vl_c_25': 49, 'vl_c_27': 46, 'vl_g_27': 114, 'pl_g_25': 0, 'pl_c_27': 0}


In [7]:
vl_g_27.gr_autoroute.size(weight="weight") / 1000, vl_c_25.gr_autoroute.size(weight="weight") / 1000

(10572.05878753935, 4988.535660391025)

In [17]:
len(vl_g_27.gr_aire_service), len(vl_c_25.gr_aire_service)

(234, 142)

In [8]:
len(vl_g_27.stations), len(vl_c_25.stations)

(489, 486)

In [9]:
vl_g_27.stations

,index,p_cum,p_max,id_station,operateur,amenageur,geometry,node_id,nature
0,0,672.0,200.0,FRPD1PSYSTOU,Power Dot France,Power Dot France,POINT (912654.027 6846194.085),2849,station_irve
1,1,2448.5,200.0,FRPD1PITMCPV,Power Dot France,Power Dot France,POINT (484572.045 6227250.773),2850,station_irve
2,3,672.0,200.0,FRPD1PSATBLT,Power Dot France,Power Dot France,POINT (1038262.052 6731018.453),2852,station_irve
3,5,2666.0,188.0,FRPD1PLCLLFL,Power Dot France,Power Dot France,POINT (470971.903 6737943.233),2854,station_irve
4,7,3000.0,250.0,FRTSLP29658,Tesla,Tesla,POINT (3854644.511 7280683.72),2856,station_irve
...,...,...,...,...,...,...,...,...,...
484,1303,3000.0,250.0,FRTSLP30237,Tesla,Tesla,POINT (3831717.622 7060810.159),4152,station_irve
485,1308,598.0,188.0,FRPD1PMBRDVZ,Power Dot France,Power Dot France,POINT (832653.337 6463299.806),4157,station_irve
486,1312,3000.0,250.0,FRTSLP29873,Tesla,Tesla,POINT (3878507.184 7775438.767),4161,station_irve
487,1313,752.0,200.0,FRPD1PACCCLM,Power Dot France,Power Dot France,POINT (925237.639 6503618.314),4162,station_irve


In [13]:
stat_g = vl_g_27.gs_all.to_geopandas_nodelist()
stat_g['operateur'].unique()

array(['Power Dot France'], dtype=object)

In [15]:
len(vl_g_27.gs_all), len(vl_g_27.gs_station), len(vl_g_27.gs_pre_station), len(vl_g_27.gs_externe)

(114, 0, 3, 111)

In [16]:
len(vl_c_25.gs_all), len(vl_c_25.gs_station), len(vl_c_25.gs_pre_station), len(vl_c_25.gs_externe)

(49, 0, 0, 49)

In [ ]:
scenar = 'vl_g_27'
scenar = 'vl_c_25'
#scenar = 'vl_c_27'
#scenar = 'pl_g_25'
#scenar = 'pl_c_27'

afir = scenarios[scenar]
afir_name = scenar

## Exemples d'utilisation

In [ ]:
afir.stations

In [ ]:
len(afir.reseau)

### Trajet entre deux stations
Exemple entre la station '4128' (Bollene) et la station '4148' (saint-martin de crau)

In [ ]:
reseau = afir.reseau
bollene = [node for node in reseau.nodes if reseau.nodes[node].get('id_station') == 'FR*IZF*EFAST*95*1*3']
bollene = bollene[0] if bollene else None
st_martin_de_crau = [node for node in reseau.nodes if reseau.nodes[node].get('id_station') == 'FR*PVD*EVG*SMC13*D01*1']
st_martin_de_crau = st_martin_de_crau[0] if st_martin_de_crau else None
trajet = bollene and st_martin_de_crau
print('trajet : ', bollene, st_martin_de_crau)

In [ ]:
if trajet:
    path = afir.gp_trajet(source=bollene, target=st_martin_de_crau, graph=False)
    print('distance entre les stations : ', nx.path_weight(reseau, path, 'weight'))
    gp_path = afir.gp_trajet(source=bollene, target=st_martin_de_crau)
    print('trajet : ', gp_path)

### Tronçons avec un faible maillage 

Identification des tronçons avec maillage insuffisant:
- un tronçon est non maillé s'il existe un point de ce tronçon situé à une distance de la plus proche station supérieure à un seuil,
- un tronçon est peu maillé s'il est sur la même branche (ensemble de tronçons entre deux stations disponibles ou bifurcations) qu'un tronçon maillé.

La station '4501' (gap) est considéré pour cet exemple comme indisponible.

In [ ]:
for afir_name, afir_sce in scenarios.items():
    indic_n = afir_sce.indicateur_afir(etendu=False)
    indic_p = afir_sce.indicateur_afir(etendu=True)
    print(afir_name, "pourcentage atteint : ", indic_p['ratio afir'] * 100, 'mini : ', indic_n['ratio afir'] * 100)
    print("distance totale, restante : ", afir_sce.size_rtet / 1000, indic_p['distance restante'] / 1000)

## Exemples de restitutions

In [ ]:
param_exp_gs = {'e_tooltip': ["source", "target"], 'e_popup': ['nature', 'weight', 'source', 'target'], 'e_color': 'grey',
                'n_name': 'station', 'e_name': 'liaison station', 'n_popup': ['amenageur', 'operateur', 'p_cum', 'p_max', 'node_id', 'id_station'], 
                'n_tooltip': "amenageur", 'n_color': 'green', 'n_marker_kwds': {'radius': 4, 'fill': True}}

param_exp_gr = {'e_popup': ['weight', 'source', 'target', 'core', "CORRIDORS", "INTROADNUM", "NATIONALRO", "ID", "nature"], 
                'n_popup': ['roadname', 'nom', 'node_id'],
                'e_tooltip': ["source", "target", "nature", "core"], 
                'n_tooltip': ["node_id", "nature"], 'n_marker_kwds': {'radius': 1, 'fill': False}}

### restitution du maillage

In [ ]:
gr_non_maille = afir.gr_non_maille(etendu=False, dispo='dispo')
gr_peu_maille = afir.gr_non_maille(etendu=True, dispo='dispo')

In [ ]:
param_sat = {'e_name': 'non_mailles', 'e_color': 'red'}
param_ext_sat = {'e_name': 'peu_mailles', 'e_color': 'orange'}

carte = afir.gs_all.explore(refmap=refnat, **param_exp_gs)
carte = afir.gr_all.explore(refmap=carte, **param_exp_gr)
carte = gr_peu_maille.explore(refmap=carte, nodes=False, **param_ext_sat)
carte = gr_non_maille.explore(refmap=carte, layercontrol=True, nodes=False, **param_sat)
carte.save(afir_name + '.html')
carte

### restitution d'un trajet

In [ ]:
#param_exp_gr = {'e_name': 'edges', 'n_name': 'nodes', 'e_popup': ['weight', 'source', 'target'], 'e_tooltip': ["source", "target"], 
#                'n_tooltip': ["node_id", "nature"], 'n_marker_kwds': {'radius': 1, 'fill': False}}
param_exp_path = {'e_name': 'path', 'e_color': 'red'}
refmap = {'tiles': 'cartodbpositron', 'location': [43.8, 5], 'zoom_start': 9}

if trajet:    
    carte = afir.gs_all.explore(refmap=refmap, **param_exp_gs)
    carte = afir.reseau.explore(refmap=carte, n_color='black', e_name='edges rtet', n_name='nodes rtet', **param_exp_gr)
    carte = gp_path.explore(refmap=carte, layercontrol=True, nodes=False, **param_exp_path)
carte

### restitution du réseau RTE-T global et central

In [ ]:
param_exp_gr = {'e_popup': ['weight', 'source', 'target', 'NATIONALRO', 'ID', 'nature', 'core'], 'e_tooltip': ["source", "target", "nature", "core"], 
                'n_tooltip': ["node_id"], 'n_marker_kwds': {'radius': 1, 'fill': False}}


carte = afir.gr_rtet_global.explore(refmap=refnat, e_color='cyan', e_name='edges global', n_name='nodes global',  n_color='black', **param_exp_gr)
carte = afir.gr_rtet_central.explore(refmap=carte, n_color='black', e_name='edges central', n_name='nodes central', layercontrol=True, **param_exp_gr)
carte

### restitution du réseau autoroute

In [ ]:
param_exp_gr = {'e_popup': ['weight', 'source', 'target', 'NATIONALRO', 'ID', 'nature'], 'e_tooltip': ["source", "target", "nature"], 
                'n_tooltip': ["node_id"], 'n_marker_kwds': {'radius': 1, 'fill': False}}



carte = afir.gr_hors_autoroute.explore(refmap=refnat, e_color='green', e_name='edges hors autoroute', n_name='nodes hors autoroute', n_color='black', **param_exp_gr)
carte = afir.gr_autoroute.explore(refmap=carte, n_color='black', e_name='edges autoroute', n_name='nodes autoroute', layercontrol=True, **param_exp_gr)
carte

### Restitution des noeuds du réseau RTE-T

In [ ]:
param_exp_gr = {'e_popup': ['weight', 'source', 'target', "CORRIDORS", "INTROADNUM", "NATIONALRO", "ID", "nature", "core"], 
                'n_popup': ['roadname', 'nom', 'node_id'],
                'e_tooltip': ["source", "target", "nature", "core"], 
                'n_tooltip': ["node_id", "nature"], 'n_marker_kwds': {'radius': 1, 'fill': False}}
param_exp_as = {'n_name': 'aire de service', 'n_color': 'red', 'n_marker_kwds': {'radius': 3, 'fill': True}}
param_exp_ech = {'n_name': 'echangeur', 'n_color': 'blue', 'n_marker_kwds': {'radius': 3, 'fill': True}}
param_exp_rp = {'n_name': 'rond-point', 'n_color': 'purple', 'n_marker_kwds': {'radius': 3, 'fill': True}}

carte = afir.gr_rond_point.explore(refmap=refnat, edges=False, **param_exp_rp)
carte = afir.gr_echangeur.explore(refmap=carte, edges=False, **param_exp_ech)
carte = afir.gr_aire_service.explore(refmap=carte, edges=False, **param_exp_as)
carte = afir.gr_hors_autoroute.explore(refmap=carte, e_color='green', e_name='edges hors autoroute', n_name='nodes hors autoroute', n_color='black', **param_exp_gr)
carte = afir.gr_autoroute.explore(refmap=carte, n_color='black', e_name='edges autoroute', n_name='nodes autoroute', layercontrol=True, **param_exp_gr)
carte

### Restitution du réseau des stations IRVE

- vert: aires de service
- rouge: aires de recharge
- violet: stations extérieures

In [ ]:
param_exp_nd = {'n_marker_kwds': {'radius': 3, 'fill': True}}
param_exp_gs = {'e_tooltip': ["source", "target"], 'e_popup': ['type', 'weight', 'source', 'target'], 'e_color': 'grey',
                'n_name': 'station', 'e_name': 'liaison station', 'n_popup': ['amenageur', 'operateur', 'p_cum', 'p_max', 'node_id', 'id_station'], 
                'n_tooltip': "amenageur", 'n_color': 'green', 'n_marker_kwds': {'radius': 4, 'fill': True}}
param_exp_gr = {'e_name': 'edges', 'n_name': 'nodes', 
                'e_popup': ['weight', 'source', 'target', 'core', 'nature', "CORRIDORS", "INTROADNUM", "NATIONALRO"], 
                'e_tooltip': ["source", "target"], 'n_tooltip': ["node_id", "nature"], 'n_marker_kwds': {'radius': 1, 'fill': False}}

carte = afir.gs_station.explore(refmap=refnat, n_color='green', n_name='aire de service', **param_exp_nd)
carte = afir.gs_pre_station.explore(refmap=carte, n_color='red', n_name='aire de recharge', **param_exp_nd)
carte = afir.gs_externe.explore(refmap=carte, n_color='purple', n_name='stations extérieures', **param_exp_nd)
carte = afir.reseau.explore(refmap=carte, layercontrol=True, n_color='black', **param_exp_gr)
carte